# Compare the mondo articles and evidence against Pubmed terms api

In [2]:
import json
import urllib
from urllib import request
from urllib import parse
import neo4j
from neo4j import GraphDatabase, ServiceUnavailable, basic_auth
import datetime
from datetime import date
import time
import logging, logging.config
import itertools
from itertools import islice
import pandas
import jmespath
import xml.etree.ElementTree as ET
import re



### Function to get a dictionary of Mondo and Gard disease IDs

In [2]:
def get_Mondo_and_GARD_diseases_list():
    """Returns list of the Mondo and GARD Diseases"""
    
    with GraphDatabase.driver("bolt://disease.ncats.io:80") as driver:
        try:
            with driver.session() as session:
                cypher_query = '''
                match p = (d:DATA)-[:PAYLOAD]->(m:S_MONDO)<-[r:R_equivalentClass]-(n:S_GARD)<-[:PAYLOAD]-(d1:DATA) 
                RETURN d.notation as MONDO_ID, d1.gard_id as GARD_ID
                '''
                results = session.run(cypher_query, parameters={})
                myData = {}
                for record in results:
                    myData[record['MONDO_ID']] = record['GARD_ID']
                    
            return myData
        except ServiceUnavailable as e:
                print(e)

mondo_and_gard_diseases = get_Mondo_and_GARD_diseases_list()

### Function to get a Gard disease IDs and disease names

In [4]:
def get_GARD_diseases_list():
        """Returns list of the GARD Diseases"""
        
        with GraphDatabase.driver("bolt://disease.ncats.io:80") as driver:
                try:
                        with driver.session() as session:
                                cypher_query = '''
                                match p = (d:DATA)-[]-(m:S_GARD) return distinct d.gard_id, d.name
                                '''
                                results = session.run(cypher_query, parameters={})
                                myData = {}
                                for record in results:
                                        myData[record['d.gard_id']] = record['d.name']
                                
                        return myData
                except ServiceUnavailable as e:
                        print(e)

gard_diseases = get_GARD_diseases_list()

### Function to get Gard Disease IDs, Disease Names and Mesh Terms

                                match (d:DATA)-[]-(g:S_GARD)-[r:N_Name]-(m:S_MESH) 
                                where d.name is not null
                                return distinct d.gard_id, d.name, Collect(r.value) as MeshTerms 


In [5]:
def get_GARD_diseases_and_MeshTerms_list():
        """Returns list of the GARD Diseases"""
        
        with GraphDatabase.driver("bolt://disease.ncats.io:80") as driver:
                try:
                        with driver.session() as session:
                                cypher_query = '''
                                match (d:DATA)-[]-(g:S_GARD)
                                optional match (g)-[r:N_Name]-(m:S_MESH) 
                                where d.name is not null
                                return distinct d.gard_id, d.name, Collect(r.value) as MeshTerms 
                                '''
                                results = session.run(cypher_query, parameters={})
                                myData = {}
                                for record in results:
                                        myData[record['d.gard_id']] = (record['d.name'], record['MeshTerms'])
                                
                        return myData
                except ServiceUnavailable as e:
                        print(e)

gard_diseases_and_MeshTerms = get_GARD_diseases_and_MeshTerms_list()

In [7]:
gard_diseases_and_MeshTerms.get('GARD:0012155')[1]

[]

### Function to get OMIM IDs and GARD IDs

In [3]:
def get_OMIM_and_GARD_diseases_list():
    """Returns list of the OMIM and GARD Diseases"""
    
    with GraphDatabase.driver("bolt://disease.ncats.io:80") as driver:
        try:
            with driver.session() as session:
                cypher_query = '''
                match (d:DATA)-[]-(m:S_GARD)
                where exists(d.gard_id)
                unwind d.xrefs as x
                return distinct d.gard_id, d.name, x
                '''
                results = session.run(cypher_query, parameters={})
                myData = {}
                for record in results:
                    if (record['x'].startswith("OMIM")):
                        #myData[record['x']] = record['d.gard_id']
                        myData[record['d.gard_id']] = record['x']
            return myData
        except ServiceUnavailable as e:
                print(e)

OMIM_and_gard_diseases = get_OMIM_and_GARD_diseases_list()

In [7]:
a = OMIM_and_gard_diseases.values()
print (len(a))


3375


### Function to get Orpha ids and Gard Ids

In [4]:
def get_Orpha_and_GARD_diseases_list():
    """Returns list of the Orpha and GARD Diseases"""
    
    with GraphDatabase.driver("bolt://disease.ncats.io:80") as driver:
        try:
            with driver.session() as session:
                cypher_query = '''
                match (d:DATA)-[]-(m:S_GARD)
                where exists(d.gard_id)
                unwind d.xrefs as x
                return distinct d.gard_id, d.name, x
                '''
                results = session.run(cypher_query, parameters={})
                myData = {}
                for record in results:
                    if (record['x'].startswith("ORPHA:")):
                        #myData[record['x']] = record['d.gard_id']
                        myData[record['d.gard_id']] = record['x']
            return myData
        except ServiceUnavailable as e:
                print(e)

Orpha_and_gard_diseases = get_Orpha_and_GARD_diseases_list()

In [ ]:
"//JDBOR/DisorderList/Disorder/OrphaCode[text()=166024]/following-sibling::PrevalenceList/Prevalence/Source[contains(.,'PMID')]"

In [15]:
def fetch_orpha_epidemiology():
    orphaFiles = "http://www.orphadata.org/data/xml/en_product9_prev.xml"
    epifile = urllib.request.urlretrieve(orphaFiles)
    print(epifile)
    tree = ET.parse(epifile[0])
    root = tree.getroot()

    for orphaid in Orpha_and_gard_diseases.values():
        elements=root.findall("./DisorderList/Disorder/[OrphaCode='" + "166024" + "']/./PrevalenceList/Prevalence/Source")
        for element in elements:
            print(element.text)
    

fetch_orpha_epidemiology()



90[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689990[PMID]_ [EXPERT]
ORPHANET_11389160[PMID]_9689990[PMID]
11389160[PMID]_9689

KeyboardInterrupt: 

### Combine Gard, Mondo and Omim

In [9]:
mondo_gard_omim_list = []
for i, (mid,gid) in enumerate(mondo_and_gard_diseases.items()):
    mondo_gard_omim_list.append((mid,gid, OMIM_and_gard_diseases.get(gid)))
    #print(i,mid,gid,OMIM_and_gard_diseases.get(gid))
    #if (i == 5):
    #    break
print (mondo_gard_omim_list)

&#39;, None), (&#39;MONDO:0018394&#39;, &#39;GARD:0012514&#39;, None), (&#39;MONDO:0017287&#39;, &#39;GARD:0012521&#39;, None), (&#39;MONDO:0017140&#39;, &#39;GARD:0012524&#39;, None), (&#39;MONDO:0017630&#39;, &#39;GARD:0012525&#39;, None), (&#39;MONDO:0010569&#39;, &#39;GARD:0012526&#39;, &#39;OMIM:304100&#39;), (&#39;MONDO:0011910&#39;, &#39;GARD:0012527&#39;, &#39;OMIM:607801&#39;), (&#39;MONDO:0021018&#39;, &#39;GARD:0012528&#39;, &#39;OMIM:603511&#39;), (&#39;MONDO:0018098&#39;, &#39;GARD:0012529&#39;, None), (&#39;MONDO:0012034&#39;, &#39;GARD:0012530&#39;, &#39;OMIM:608423&#39;), (&#39;MONDO:0012193&#39;, &#39;GARD:0012531&#39;, &#39;OMIM:609115&#39;), (&#39;MONDO:0013297&#39;, &#39;GARD:0012532&#39;, &#39;OMIM:613530&#39;), (&#39;MONDO:0011787&#39;, &#39;GARD:0012533&#39;, &#39;OMIM:607155&#39;), (&#39;MONDO:0012127&#39;, &#39;GARD:0012534&#39;, &#39;OMIM:608807&#39;), (&#39;MONDO:0012248&#39;, &#39;GARD:0012535&#39;, &#39;OMIM:609308&#39;), (&#39;MONDO:0012652&#39;, &#39;GARD

### Function to get mondo articles

In [10]:
def find_mondo_articles(MondoID):
    """fetch mondo articles and return a map"""
    # https://api.monarchinitiative.org/api/bioentity/disease/MONDO:0007846/publications
    data = request.urlopen("https://api.monarchinitiative.org/api/bioentity/disease/" + MondoID + "/publications").read().decode()
    return json.loads(data)




### Function to get pubmed articles

In [11]:
def find_articles(keyword, mindate, maxdate):
        """fetch articles and return a map"""        
        params = urllib.parse.urlencode({'db': 'pubmed', 'term':keyword, 'mindate':mindate, 'maxdate':maxdate, 'retmode': 'json', 'retmax':"1000"})
        data = request.urlopen("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?%s" % params).read().decode()
        #print ("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?%s" % params)
        return json.loads(data)

### Function to get pubmed articles but the term is quoted

In [12]:
def find_articles_quoted(keyword, mindate, maxdate):
        """fetch articles and return a map"""        
        params = urllib.parse.urlencode({'db': 'pubmed', 'term': '"'+keyword+'"', 'mindate':mindate, 'maxdate':maxdate, 'retmode': 'json', 'retmax':"1000"})
        data = request.urlopen("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?%s" % params).read().decode()
        #print ("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?%s" % params)
        return json.loads(data)

### Function to get pubmed articles by mesh terms

In [13]:
def find_articles_by_mesh(meshterms, mindate, maxdate):
        """fetch articles and return a map"""        
        params = urllib.parse.urlencode({'db': 'pubmed', 'term': meshterms, 'mindate':mindate, 'maxdate':maxdate, 'retmode': 'json', 'retmax':"1000"})
        url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?%s" % params
        print(url)
        data = request.urlopen(url).read().decode()
        #print ("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?%s" % params)
        return json.loads(data)

query = ''
for mesh in gard_diseases_and_MeshTerms.get('GARD:0013215')[1]:
    print(mesh)
    query = query + '"' + mesh + '"[MeSH Terms] OR ' 
query = query[:-3]
print(query)
gard_test = find_articles_by_mesh(query,"1900/01/01","2020/10/23")

ADENOCARCINOMA OF KIDNEY
RENAL CELL ADENOCARCINOMA
RENAL CELL CARCINOMA
&quot;ADENOCARCINOMA OF KIDNEY&quot;[MeSH Terms] OR &quot;RENAL CELL ADENOCARCINOMA&quot;[MeSH Terms] OR &quot;RENAL CELL CARCINOMA&quot;[MeSH Terms] 
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&amp;term=%22ADENOCARCINOMA+OF+KIDNEY%22%5BMeSH+Terms%5D+OR+%22RENAL+CELL+ADENOCARCINOMA%22%5BMeSH+Terms%5D+OR+%22RENAL+CELL+CARCINOMA%22%5BMeSH+Terms%5D+&amp;mindate=1900%2F01%2F01&amp;maxdate=2020%2F10%2F23&amp;retmode=json&amp;retmax=1000


### Function to get OMIM articles

omim.entryList[0].entry.referenceList[*].reference.[referenceNumber, pubmedID]

({.*?}|{.*?:.*?})

In [3]:
def find_OMIM_articles(OMIMNumber):
        """fetch articles and return a map"""        
        params = urllib.parse.urlencode({'mimNumber': OMIMNumber, 'include':"all", 'format': 'json', 'apiKey':"wr20AKemSkWnoFR8b-vSkA"})
        data = request.urlopen("https://api.omim.org/api/entry?%s" % params).read().decode()
        return json.loads(data)

omim_reference = find_OMIM_articles("300100")
import jmespath
pmids = jmespath.search("omim.entryList[0].entry.referenceList[*].reference.pubmedID",omim_reference)
refNumbers = jmespath.search("omim.entryList[0].entry.referenceList[*].reference.[referenceNumber,pubmedID]",omim_reference)
textSections = jmespath.search("omim.entryList[0].entry.textSectionList[*].textSection",omim_reference)
for refNumber,pmid in refNumbers:
        print (refNumber, pmid)
        for t in textSections:
                refs = re.findall("({.*?}|{.*?:.*?})",t['textSectionContent'])
                refs
                if (refs is not None):
                        sectionReferenced = []
                        for ref in refs:
                                splitRef= ref[1:].split(":")
                                if (len(splitRef)>1):
                                        if (not re.search(",",splitRef[0])):
                                                sectionReferenced.append(splitRef[0])
                                        else:
                                                multiples =  splitRef[0].split(",")
                                                for r in multiples:
                                                        sectionReferenced.append(r)

                print(t['textSectionName'], refs)
                print(t['textSectionName'], sectionReferenced)
                print("Unique Values for ",t['textSectionName'], set(sectionReferenced))
                
        if (refNumber > 1): break


1 None
text ['{300371}']
text []
Unique Values for  text set()
description ['{97:Moser (1997)}', '{89:Moser et al. (2005)}']
description ['97', '89']
Unique Values for  description {'97', '89'}
Found a multiple 102,103
Found a multiple ['102', '103']
clinicalFeatures ['{84:Moser et al. (2000)}', '{29:Davis et al. (1979)}', "{103:O'Neill et al. (1982)}", "{102,103:O'Neill et al. (1980, 1982)}", '{13:Berg et al. (1989)}', '{156:Willems et al. (1990)}', '{57:Holmberg et al. (1991)}', '{139:Sobue et al. (1994)}', '{66:Korenke et al. (1996)}', '{31:Di Rocco et al. (2001)}', '{155:Wilichowski et al. (1998)}', '{66:Korenke et al. (1996)}', '{31:Di Rocco et al. (2001)}', '{28:Cox et al. (2006)}', '{28:Cox et al. (2006)}', '{131:Schaumburg et al., 1975}', '{6:Aubourg et al., 1982}', '{137:Siemerling and Creutzfeldt (1923)}', '{23:Budka et al. (1976)}', '{45:Griffin et al. (1977)}', '{130:Schaumburg et al. (1977)}', "{105:O'Neill et al. (1985)}", '{27:Cotrufo et al. (1987)}', '{145:Uyama et al. 

### Code to build a dataframe of mondo, gard and omim comparisons

In [17]:
def isInt(s):
    try:     i = int(s)
    except:  return False
    return True

#setup the data frame
df = pandas.DataFrame(columns=['gardid','mondoid','omimid','disease', 'MeshTerms', 'QueryTranslation',
    'Eutils pmids Count','Mondo pmids Count','OMIM pmids Count',
    'Eutils Mondo Intersection Count','Eutils - Mondo Count','Mondo - Eutils Count',
    'Eutils OMIM Intersection Count','Eutils - OMIM Count','OMIM - Eutils Count',  
    'Eutils pmids','Mondo pmids','OMIM pmids',
    'Eutils Mondo Intersection','Eutils - Mondo','Mondo - Eutils',
    'Eutils OMIM Intersection','Eutils - OMIM','OMIM - Eutils'])

for i,(mid, gid, oid) in enumerate(mondo_gard_omim_list):
    print (i,mid, gid, oid)
    #if (i>50): break
    time.sleep(0.3)

    #get pmids from both eutils and mondo
    pubmed_result = find_articles(gard_diseases[gid],"1900/01/01","2020/10/23")
    pubmed_pmids = jmespath.search("esearchresult.idlist",pubmed_result)
    pubmed_pmids = [int(pmid) for pmid in pubmed_pmids]

    pubmed_querytranslation = jmespath.search("esearchresult.querytranslation", pubmed_result)

    mondo_result = find_mondo_articles(mid)
    mondo_pmids_search = jmespath.search("associations[*].object.id",mondo_result)
    mondo_pmids = [int(str.split(pmid,":")[1]) for pmid in mondo_pmids_search if isInt(str.split(pmid,":")[1])]


    omim_pmids = []
    if (oid is not None):
        oidNumber = str.split(oid,sep=":")[1]
        omim_result = find_OMIM_articles(oidNumber)
        omim_pmids = jmespath.search("omim.entryList[0].entry.referenceList[*].reference.pubmedID",omim_result)

    #build sets for what's common, and only in Eutils, and only in Mondo
    isection = em = me = set()
    if (pubmed_pmids is not None and mondo_pmids is not None):
        isection = set.intersection(set(pubmed_pmids),set(mondo_pmids))
        em = set.difference(set(pubmed_pmids),set(mondo_pmids))
        me = set.difference(set(mondo_pmids),set(pubmed_pmids))

    eutil_omim_intersection = eo = oe = set()
    if (pubmed_pmids is not None and omim_pmids is not None):
        eutil_omim_intersection = set.intersection(set(pubmed_pmids),set(omim_pmids))
        eo = set.difference(set(pubmed_pmids),set(omim_pmids))
        oe = set.difference(set(omim_pmids),set(pubmed_pmids))

    #save the data to a frame
    df = df.append({'gardid':gid,'mondoid':mid,'omimid':oid,'disease':gard_diseases[gid], 'MeshTerms': gard_diseases_and_MeshTerms.get(gid)[1], 'QueryTranslation':pubmed_querytranslation,
    'Eutils pmids Count':len(pubmed_pmids) if (pubmed_pmids is not None) else None,'Mondo pmids Count':len(mondo_pmids) if (mondo_pmids is not None) else None,'OMIM pmids Count':len(omim_pmids) if (omim_pmids is not None) else None, 
    'Eutils Mondo Intersection Count':len(isection),'Eutils - Mondo Count':len(em),'Mondo - Eutils Count':len(me), 
    'Eutils OMIM Intersection Count':len(eutil_omim_intersection),'Eutils - OMIM Count':len(eo),'OMIM - Eutils Count':len(oe),
    'Eutils pmids':pubmed_pmids,'Mondo pmids':mondo_pmids,'OMIM pmids':omim_pmids,
    'Eutils Mondo Intersection':isection,'Eutils - Mondo':em,'Mondo - Eutils':me,
    'Eutils OMIM Intersection':eutil_omim_intersection,'Eutils - OMIM':eo,'OMIM - Eutils':oe},ignore_index=True)
    
    #save data to file every 5 iterations in case process crashes
    if (i % 5 == 0):
        df.to_csv("mondo_gard_omim_comparison.csv")

#save data to file once done
df.to_csv("mondo_gard_omim_comparison.csv")

1071 GARD:0011963 None
5050 MONDO:0001023 GARD:0011965 None
5051 MONDO:0016377 GARD:0011967 None
5052 MONDO:0023149 GARD:0011970 None
5053 MONDO:0019105 GARD:0011971 None
5054 MONDO:0020250 GARD:0011972 None
5055 MONDO:0004977 GARD:0011973 None
5056 MONDO:0012269 GARD:0011974 OMIM:609425
5057 MONDO:0020640 GARD:0011979 None
5058 MONDO:0012514 GARD:0011980 OMIM:610532
5059 MONDO:0018904 GARD:0011982 None
5060 MONDO:0018338 GARD:0011983 OMIM:615513
5061 MONDO:0017366 GARD:0011984 None
5062 MONDO:0020469 GARD:0011985 None
5063 MONDO:0025510 GARD:0011989 None
5064 MONDO:0017617 GARD:0011992 None
5065 MONDO:0005130 GARD:0011998 None
5066 MONDO:0011340 GARD:0012008 OMIM:603569
5067 MONDO:0008867 GARD:0012010 None
5068 MONDO:0013758 GARD:0012011 OMIM:614455
5069 MONDO:0016611 GARD:0012015 None
5070 MONDO:0016747 GARD:0012016 None
5071 MONDO:0002073 GARD:0012017 None
5072 MONDO:0005089 GARD:0012018 None
5073 MONDO:0006939 GARD:0012020 None
5074 MONDO:0007022 GARD:0012021 None
5075 MONDO:000260

### Code to build a dataframe of mondo and gard disease comparisons

In [9]:
#setup the data frame
df = pandas.DataFrame(columns=['gardid','mondoid','disease', 'Eutils pmids Count','Mondo pmids Count','Intersection Count','Eutils - Mondo Count','Mondo - Eutils Count',  'Eutils pmids','Mondo pmids','Intersection','Eutils - Mondo','Mondo - Eutils'])

#iterate throught the list of mondo and gard diseases
for i, (mid,gid) in enumerate(mondo_and_gard_diseases.items()):
    print(i,mid,gid)
    
    #lets only grab the first x amount
    #if (i>=1000): break
    
    #lets pause a lil bit so that pubmed and mondo apis don't get mad at us 
    time.sleep(0.3)
    
    #get pmids from both eutils and mondo
    pubmed_result = find_articles(gard_diseases[gid],"1900/01/01","2020/09/25")
    mondo_result = find_mondo_articles(mid)
    
    #extract the pmids
    if ('esearchresult' in pubmed_result and 'count' in pubmed_result['esearchresult'] 
            and int(pubmed_result['esearchresult']['count'])>0):
            pubmed_ids = [pmid for pmid in pubmed_result['esearchresult']['idlist']]
    
    if ('associations' in mondo_result and len(mondo_result['associations']) > 0):
        mondo_ids = [str.split(association['object']['id'],":")[1] for association in mondo_result['associations'] if('object' in association and 'id' in association['object'] and str.split(association['object']['id'],":")[0] == 'PMID')] 

    #build sets for what's common, and only in Eutils, and only in Mondo
    isection = set.intersection(set(pubmed_ids),set(mondo_ids))
    em = set.difference(set(pubmed_ids),set(mondo_ids))
    me = set.difference(set(mondo_ids),set(pubmed_ids))

    #save the data to a frame
    df = df.append({'gardid':gid,'mondoid':mid,'disease':gard_diseases[gid], 'Eutils pmids Count':len(pubmed_ids),'Mondo pmids Count':len(mondo_ids),'Intersection Count':len(isection),'Eutils - Mondo Count':len(em),'Mondo - Eutils Count':len(me),  'Eutils pmids':pubmed_ids,'Mondo pmids':mondo_ids,'Intersection':isection,'Eutils - Mondo':em,'Mondo - Eutils':me},ignore_index=True)
    
    #save data to file every 5 iterations in case process crashes
    if (i % 5 == 0):
        df.to_csv("mondo_and_gard_comparison.csv")

#save data to file once done
df.to_csv("mondo_and_gard_comparison.csv")

4918 MONDO:0003658 GARD:0010897
4919 MONDO:0020338 GARD:0010898
4920 MONDO:0000507 GARD:0010899
4921 MONDO:0008632 GARD:0010901
4922 MONDO:0016987 GARD:0010902
4923 MONDO:0015262 GARD:0010903
4924 MONDO:0023388 GARD:0010904
4925 MONDO:0015780 GARD:0010905
4926 MONDO:0015999 GARD:0010906
4927 MONDO:0010455 GARD:0010907
4928 MONDO:0011979 GARD:0010909
4929 MONDO:0018929 GARD:0010910
4930 MONDO:0015175 GARD:0010911
4931 MONDO:0043352 GARD:0010912
4932 MONDO:0018306 GARD:0010913
4933 MONDO:0012126 GARD:0010914
4934 MONDO:0010420 GARD:0010915
4935 MONDO:0010385 GARD:0010916
4936 MONDO:0012905 GARD:0010917
4937 MONDO:0013858 GARD:0010919
4938 MONDO:0015032 GARD:0010921
4939 MONDO:0016854 GARD:0010922
4940 MONDO:0011424 GARD:0010924
4941 MONDO:0019378 GARD:0010925
4942 MONDO:0016168 GARD:0010927
4943 MONDO:0008890 GARD:0010928
4944 MONDO:0018835 GARD:0010929
4945 MONDO:0001427 GARD:0010930
4946 MONDO:0013254 GARD:0010933
4947 MONDO:0013607 GARD:0010934
4948 MONDO:0016838 GARD:0010935
4949 MON

In [ ]:
# Find the mesh term for each disease from meshb and run search again
# Also breakdown by publication type
# check omim publication 

